# Setup

In [83]:
import pandas as pd
from pathlib import Path

In [84]:
#input args
log_name = 'logs/LOG_008.csv'
ticks_per_second_mb = 1000
out_dir = './out'

#derived args
split_logs_dir = f'{out_dir}/split_logs/'

In [85]:
#create output folders
Path(f'{out_dir}/split_logs').mkdir(parents=True, exist_ok=True)
Path(f'{out_dir}/plots').mkdir(parents=True, exist_ok=True)

# Full Log

In [86]:
full_log = pd.read_csv(log_name, delimiter = ';')
full_log.index.rename('Log Entry', inplace = True)
full_log.head()

,Timestamp,Log Entry Type,Data
Log Entry,,,
0,601,1,"1,3,0,0,0,0,0,0,0,0,0,0"
1,601,1,"2,3,0,0,0,0,0,0,0,0,0,0"
2,602,1,"3,3,0,0,0,0,0,0,0,0,0,0"
3,603,1,"1,3,0,0,0,0,0,0,0,0,0,0"
4,603,1,"2,3,0,0,0,0,0,0,0,0,0,0"


In [87]:
#convert timestamp ticks to seconds
full_log['Time_s'] = full_log['Timestamp']/ticks_per_second_mb

In [88]:
#change entry type ids to enums
full_log['Log Entry Type'].replace({1:'SENSOR', 2:'STATE', 3:'ESTIMATOR_VAR', 4: 'CONTROLLER_OUTPUT', 5:'MOTOR', 6:'MSG'}, inplace = True)

# Sensor Log

In [89]:
#extract sensor entries
sensor_log = full_log[full_log['Log Entry Type'] == 'SENSOR'].copy()
#extract data for sensor entries
sensor_log[['Sensor Board ID', 'Sensor Type', 'Data']] = sensor_log['Data'].str.split(',', expand = True, n = 2)

#change sensor type ids to enums
sensor_log['Sensor Type'] = sensor_log['Sensor Type'].astype(int)
sensor_log['Sensor Type'].replace({1:'BARO', 2:'IMU', 3:'GPS', 4: 'BATTERY'}, inplace = True)

In [90]:
#extract individual sensor types
baro_log = sensor_log[sensor_log['Sensor Type'] == 'BARO'].copy()
imu_log = sensor_log[sensor_log['Sensor Type'] == 'IMU'].copy()
gps_log = sensor_log[sensor_log['Sensor Type'] == 'GPS'].copy()
battery_log = sensor_log[sensor_log['Sensor Type'] == 'BATTERY'].copy()

## Baro Log

In [91]:
#extract data for baro sensors
baro_log[['Pressure', 'Temperature', 'Sensor Board Timestamp']] = baro_log['Data'].str.split(',', expand = True)
baro_log.drop(columns = ['Data'], inplace = True)
#reorder columns
baro_log = baro_log[['Timestamp', 'Time_s', 'Log Entry Type', 'Sensor Board ID', 'Sensor Board Timestamp', 'Sensor Type', 'Pressure', 'Temperature']]

#convert temperature to Celsius
fmt_tmp = lambda x: x / 100
baro_log['Temperature'] = baro_log['Temperature'].apply(pd.to_numeric).apply(fmt_tmp)
baro_log.head()

,Timestamp,Time_s,Log Entry Type,Sensor Board ID,Sensor Board Timestamp,Sensor Type,Pressure,Temperature
Log Entry,,,,,,,,
6,811,0.811,SENSOR,1,0,BARO,0,0.00
8,811,0.811,SENSOR,2,0,BARO,0,0.00
10,811,0.811,SENSOR,3,0,BARO,0,0.00
12,821,0.821,SENSOR,1,0,BARO,0,0.00
14,821,0.821,SENSOR,2,22423,BARO,95712,25.55


In [92]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
baro_log.to_csv(f'{split_logs_dir}/baro.csv')

## IMU Log

In [93]:
#generate Acc_x, Acc_y, Acc_z, Gyro_x, Gyro_y, Gyro_z
dim3 = lambda x : [x + '_' + s for s in ['x', 'y', 'z']]
imu_cols = dim3('Acc') + dim3('Gyro')

#extract data for imu sensors
imu_log[imu_cols + ['Sensor Board Timestamp']] = imu_log['Data'].str.split(',', expand = True)
imu_log.drop(columns = ['Data'], inplace = True)
#reorder columns
imu_log = imu_log[['Timestamp', 'Time_s', 'Log Entry Type', 'Sensor Board ID', 'Sensor Board Timestamp', 'Sensor Type'] + imu_cols]

#scale acceleration and rotation to float

#convert to gs
fmt_acc = lambda x: x / 1024
#convert to degrees per second
fmt_gyro = lambda x: x / 32.4

imu_log[dim3('Acc')] = imu_log[dim3('Acc')].apply(pd.to_numeric).apply(fmt_acc)
imu_log[dim3('Gyro')] = imu_log[dim3('Gyro')].apply(pd.to_numeric).apply(fmt_gyro)

imu_log.head()

,Timestamp,Time_s,Log Entry Type,Sensor Board ID,Sensor Board Timestamp,Sensor Type,Acc_x,Acc_y,Acc_z,Gyro_x,Gyro_y,Gyro_z
Log Entry,,,,,,,,,,,,
7,811,0.811,SENSOR,1,0,IMU,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
9,811,0.811,SENSOR,2,0,IMU,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
11,811,0.811,SENSOR,3,0,IMU,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
13,821,0.821,SENSOR,1,0,IMU,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
15,821,0.821,SENSOR,2,22414,IMU,0.010742,-0.024414,-0.99707,0.123457,-0.493827,-0.277778


In [94]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
imu_log.to_csv(f'{split_logs_dir}/imu.csv')

## GPS Log

In [95]:
gps_cols = ['Hour', 'Minute', 'Second', 'Latitude [deg]', 'Latitude [decimal]', 'Longitude [deg]', 'Longitude [decimal]', 'Satellite', 'Altitude', 'HDOP']
gps_log[gps_cols] = gps_log['Data'].str.split(',', expand = True)
gps_log = gps_log[['Timestamp', 'Time_s', 'Log Entry Type', 'Sensor Board ID', 'Sensor Type', ] + gps_cols]
gps_log.head()

,Timestamp,Time_s,Log Entry Type,Sensor Board ID,Sensor Type,Hour,Minute,Second,Latitude [deg],Latitude [decimal],Longitude [deg],Longitude [decimal],Satellite,Altitude,HDOP
Log Entry,,,,,,,,,,,,,,,
0,601,0.601,SENSOR,1,GPS,0,0,0,0,0,0,0,0,0,0
1,601,0.601,SENSOR,2,GPS,0,0,0,0,0,0,0,0,0,0
2,602,0.602,SENSOR,3,GPS,0,0,0,0,0,0,0,0,0,0
3,603,0.603,SENSOR,1,GPS,0,0,0,0,0,0,0,0,0,0
4,603,0.603,SENSOR,2,GPS,0,0,0,0,0,0,0,0,0,0


In [96]:
gps_log.to_csv(f'{split_logs_dir}/gps.csv')

## Battery Log

In [97]:
battery_cols = ['Battery', 'Consumption', 'Current', 'Supply']
battery_log[battery_cols] = battery_log['Data'].str.split(',', expand = True)
battery_log = battery_log[['Timestamp', 'Time_s', 'Log Entry Type', 'Sensor Board ID', 'Sensor Type', ] + battery_cols]
battery_log.head()

,Timestamp,Time_s,Log Entry Type,Sensor Board ID,Sensor Type,Battery,Consumption,Current,Supply
Log Entry,,,,,,,,,
137,1002,1.002,SENSOR,1,BATTERY,353,0,0,3278
1054,2021,2.021,SENSOR,1,BATTERY,353,0,0,3275
1979,3041,3.041,SENSOR,1,BATTERY,358,0,0,3275
2929,4061,4.061,SENSOR,1,BATTERY,358,0,0,3275
3879,5081,5.081,SENSOR,1,BATTERY,362,0,0,3274


In [98]:
battery_log.to_csv(f'{split_logs_dir}/battery.csv')

# State Log

In [99]:
#IDLE = 1, AIRBRAKE_TEST, THRUSTING, COASTING, DESCENT, RECOVERY

In [100]:
#extract state change entries
state_log = full_log[full_log['Log Entry Type'] == 'STATE'].copy()
state_log.rename(columns={'Data': 'State'}, inplace = True)
state_log['State'] = state_log['State'].astype(int)
state_log['State'].replace({1:'IDLE', 2:'AIRBRAKE_TEST', 3:'THRUSTING', 4:'COASTING', 5:'DESCENT', 6:'RECOVERY'}, inplace = True)
state_log.head()

,Timestamp,Log Entry Type,State,Time_s
Log Entry,,,,
136,1001,STATE,IDLE,1.001
148,1011,STATE,IDLE,1.011
156,1021,STATE,IDLE,1.021
164,1031,STATE,IDLE,1.031
172,1041,STATE,IDLE,1.041


In [101]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
state_log.to_csv(f'{split_logs_dir}/state_transitions.csv')

# Estimator Variable Log

In [102]:
#extract estimator var entries
estimator_var_log = full_log[full_log['Log Entry Type'] == 'ESTIMATOR_VAR'].copy()
estimator_var_log[['World Position (z)', 'Rocket Velocity (z)',  'Rocket Acceleration (z)']] = estimator_var_log['Data'].str.split(',', expand = True)

estimator_var_log.drop(columns = ['Data'], inplace = True)
#reorder columns
estimator_var_log = estimator_var_log[['Timestamp', 'Time_s', 'Log Entry Type', 'World Position (z)', 'Rocket Velocity (z)', 'Rocket Acceleration (z)']]

fmt_est = lambda x: x / 1000
estimator_var_log[['World Position (z)', 'Rocket Velocity (z)',  'Rocket Acceleration (z)']] = estimator_var_log[['World Position (z)', 'Rocket Velocity (z)',  'Rocket Acceleration (z)']].apply(pd.to_numeric).apply(fmt_est)
estimator_var_log.head()

,Timestamp,Time_s,Log Entry Type,World Position (z),Rocket Velocity (z),Rocket Acceleration (z)
Log Entry,,,,,,
66,901,0.901,ESTIMATOR_VAR,0.0,0.001,0.188
73,911,0.911,ESTIMATOR_VAR,0.0,0.003,0.185
80,921,0.921,ESTIMATOR_VAR,0.0,0.005,0.188
87,931,0.931,ESTIMATOR_VAR,0.0,0.007,0.182
94,941,0.941,ESTIMATOR_VAR,0.0,0.009,0.185


In [103]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
estimator_var_log.to_csv(f'{split_logs_dir}/estimator_var.csv')

# Controller Output Log

In [104]:
#extract estimator var entries
controller_out_log = full_log[full_log['Log Entry Type'] == 'CONTROLLER_OUTPUT'].copy()
controller_out_log[['Controller Output', 'Reference Error', 'Integrated Error']] = controller_out_log['Data'].str.split(',', expand = True)

controller_out_log.drop(columns = ['Data'], inplace = True)
#reorder columns
controller_out_log = controller_out_log[['Timestamp', 'Time_s', 'Log Entry Type', 'Controller Output', 'Reference Error', 'Integrated Error']]

fmt_cnt = lambda x: x / 1000
controller_out_log[['Controller Output', 'Reference Error', 'Integrated Error']] = controller_out_log[['Controller Output', 'Reference Error', 'Integrated Error']].apply(pd.to_numeric).apply(fmt_cnt)
controller_out_log.head()

,Timestamp,Time_s,Log Entry Type,Controller Output,Reference Error,Integrated Error
Log Entry,,,,,,
220,1101,1.101,CONTROLLER_OUTPUT,0.0,0.0,0.0
229,1111,1.111,CONTROLLER_OUTPUT,0.0,0.0,0.0
238,1121,1.121,CONTROLLER_OUTPUT,0.0,0.0,0.0
247,1131,1.131,CONTROLLER_OUTPUT,0.0,0.0,0.0
256,1141,1.141,CONTROLLER_OUTPUT,0.0,0.0,0.0


In [105]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
controller_out_log.to_csv(f'{split_logs_dir}/controller_out.csv')

# Message Log

In [106]:
#extract message entries
msg_log = full_log[full_log['Log Entry Type'] == 'MOTOR'].copy()
msg_log.head()

,Timestamp,Log Entry Type,Data,Time_s
Log Entry,,,,
2025,3084,MOTOR,"0,0",3.084
2062,3128,MOTOR,"0,0",3.128
2099,3164,MOTOR,"0,0",3.164
2136,3204,MOTOR,"0,0",3.204
2173,3244,MOTOR,"0,0",3.244


In [107]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
msg_log.to_csv(f'{split_logs_dir}/motor.csv')

In [108]:
#extract message entries
msg_log = full_log[full_log['Log Entry Type'] == 'MSG'].copy()
msg_log.head()

,Timestamp,Log Entry Type,Data,Time_s
Log Entry,,,,


In [109]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
msg_log.to_csv(f'{split_logs_dir}/messages.csv')